In [1]:
import pandas as pd
import numpy as np
import random
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

In [2]:
!pip install implicit
import implicit

In [3]:
user_review = pd.read_csv("/content/drive/My Drive/archive/user_review_mat_food_rest.csv")

In [4]:
user_review.head()

,Unnamed: 0,user_id,business_id,user_stars,name,city,business_stars
0,0,b'V34qejxNsCbcgD8C0HVk-Q',b'HQl28KMwrEKHqhFrrDqVNQ',5.0,"b""Deagan's Kitchen & Bar""",b'Lakewood',4.0
1,1,b'zFCuveEe6M-ijY1iy23IJg',b'HQl28KMwrEKHqhFrrDqVNQ',5.0,"b""Deagan's Kitchen & Bar""",b'Lakewood',4.0
2,2,b'4V985R3RG-rv0B7WCPQzeQ',b'HQl28KMwrEKHqhFrrDqVNQ',1.0,"b""Deagan's Kitchen & Bar""",b'Lakewood',4.0
3,3,b'nFGcoL6wuPQzxsNJVSfGrA',b'HQl28KMwrEKHqhFrrDqVNQ',4.0,"b""Deagan's Kitchen & Bar""",b'Lakewood',4.0
4,4,b'CJqgUQeWhdgbDyLAFy7xvQ',b'HQl28KMwrEKHqhFrrDqVNQ',4.0,"b""Deagan's Kitchen & Bar""",b'Lakewood',4.0


In [5]:
## remove users having less than 15 ratings to avoid coldstarts ##
## need to try different 'n' and check accuracy ##
users = user_review[['user_id','business_id']].groupby('user_id').count().sort_values(by='business_id',ascending=False).reset_index()
users = users[users.business_id >= 15]

In [6]:
## joining with the user_review table to filter out the users with more than 15 reviews ##
user_review = pd.merge(user_review,users['user_id'],how='inner', on='user_id')

business_lookup = user_review[['business_id', 'name']].drop_duplicates()
business_lookup['business_id'] = business_lookup.business_id.astype(str)

user_review = user_review[['user_id','business_id','user_stars']]

In [7]:
user_review.head()

,user_id,business_id,user_stars
0,b'V34qejxNsCbcgD8C0HVk-Q',b'HQl28KMwrEKHqhFrrDqVNQ',5.0
1,b'V34qejxNsCbcgD8C0HVk-Q',b'ErxACeMyF4hSQyGhpEt4Tw',5.0
2,b'V34qejxNsCbcgD8C0HVk-Q',b'eoJfl5vG7X87QhcKb0nt5Q',4.0
3,b'V34qejxNsCbcgD8C0HVk-Q',b'VH2BjFjc9usDquDgoHhptA',4.0
4,b'V34qejxNsCbcgD8C0HVk-Q',b'0youcKV6-eE3F2MQj1l6Fw',4.0


In [8]:
len(user_review)

1641120

In [9]:
## need to remove the duplicate reviews based on date and not just the last one ##
user_review = user_review.drop_duplicates(subset=['user_id', 'business_id'], keep='last')
len(user_review)

1546995

In [10]:
users = list(np.sort(user_review.user_id.unique())) # Get unique users
business = list(user_review.business_id.unique()) # Get unique businesses
ratings = list(user_review.user_stars) # All ratings

# Get the associated row indices
rows = user_review.user_id.astype('category').cat.codes 
#rows = pd.Categorical(user_review['user_id'], categories=users)
# Get the associated column indices
cols = user_review.business_id.astype('category').cat.codes


ratings_sparse = sparse.csr_matrix((ratings, (rows, cols)), shape=(len(users), len(business)))

In [11]:
ratings_sparse

<42985x50106 sparse matrix of type '<class 'numpy.float64'>'
	with 1546995 stored elements in Compressed Sparse Row format>

In [12]:
# The matrix is very sparse which might be a problem
matrix_size = ratings_sparse.shape[0]*ratings_sparse.shape[1] # Number of possible interactions in the matrix
num_ratings = len(ratings_sparse.nonzero()[0]) # Number of businesses interacted with
sparsity = 100*(1 - (num_ratings/matrix_size))
sparsity

99.92817390677186

In [13]:
ratings_sparse.nonzero()[1].shape


(1546995,)

In [18]:
## need to make this training and test dataset better for evaluating recall and precision 
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-business matrix and "mask" a percentage of the original ratings where a
    user-business rating has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,business index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-business pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    business_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, business_inds] = 0 # Assign all of the randomly chosen user-business pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [19]:
ratings_train, ratings_test, users_business_altered = make_train(ratings_sparse, pct_test = 0.2)

In [20]:
## keep trying different values of alpha
alpha = 1
user_vecs, business_vecs = implicit.alternating_least_squares((ratings_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

In [21]:
user_vecs.shape

(42985, 32)

In [22]:
business_vecs.shape

(50106, 32)

In [23]:
# def auc_score(predictions, test):
#     '''
#     This simple function will output the area under the curve using sklearn's metrics. 
    
#     parameters:
    
#     - predictions: your prediction output
    
#     - test: the actual target result you are comparing to
    
#     returns:
    
#     - AUC (area under the Receiver Operating Characterisic curve)
#     '''
#     fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
#     return metrics.auc(fpr, tpr)   

In [24]:
# def calc_mean_auc(training_set, altered_users, predictions, test_set):
#     '''
#     This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
#     parameters:
    
#     training_set - The training set resulting from make_train, where a certain percentage of the original
#     user/item interactions are reset to zero to hide them from the model 
    
#     predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
#     These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
#     altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
#     test_set - The test set constucted earlier from make_train function
    
    
    
#     returns:
    
#     The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
#     there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
#     '''
    
    
#     store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
#     popularity_auc = [] # To store popular AUC scores
#     pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
#     item_vecs = predictions[1]
#     for user in altered_users: # Iterate through each user that had an item altered
#         training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
#         zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
#         # Get the predicted values based on our user/item vectors
#         user_vec = predictions[0][user,:]
#         pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
#         # Get only the items that were originally zero
#         # Select all ratings from the MF prediction for this user that originally had no iteraction
#         actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
#         # Select the binarized yes/no interaction pairs from the original full data
#         # that align with the same pairs in training 
#         pop = pop_items[zero_inds] # Get the item popularity for our chosen items
#         store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
#         popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
#     # End users iteration
    
#     return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
#    # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [25]:
# AUC for our recommender system, mean AUC
# calc_mean_auc(ratings_train, users_business_altered,[sparse.csr_matrix(user_vecs), sparse.csr_matrix(business_vecs.T)], ratings_test)

In [26]:
user_arr = np.array(users) # Array of user IDs from the ratings matrix
business_arr = np.array(business)

In [27]:
def get_business_visited(user_id, mf_train, user_list, business_list, business_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    user_id - Input the user's id number that you want to see prior visits of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    user_list - The array of users used in the ratings matrix
    
    business_list - The array of businesses rated in the ratings matrix
    
    business_lookup - A simple pandas dataframe of the unique business/name descriptions
    
    returns:
    
    A list of business IDs and names for a particular user that were already rated
    '''
    user_ind = np.where(user_list == user_id)[0][0] # Returns the index row of our user id
    business_ind = mf_train[user_ind,:].nonzero()[1] # Get column indices of purchased items
    business_codes = business_list[business_ind] # Get the stock codes for our purchased items
    return business_lookup.loc[business_lookup.business_id.isin(business_codes)]

In [28]:
## get user ids
user_arr[:10]

array(["b'---1lKK3aKOuomHnwAkAow'", "b'--2HUmLkcNHZp0xw6AMBPg'",
       "b'--4rAAfZnEIAKJE80aIiYg'", "b'--BumyUHiO_7YsHurb9Hkw'",
       "b'--CIuK7sUpaNzalLAlHJKA'", "b'--NIc98RMssgy0mSZL3vpA'",
       "b'--Nnm_506G_p8MxAOQna5w'", "b'--Qh8yKWAvIP4V4K8ZPfHA'",
       "b'--RlSfc-QmcHFGHyX6aVjA'", "b'--UOvCH5qEgdNQ8lzR8QYQ'"],
      dtype='<U25')

In [29]:
## businesses visited by the user
get_business_visited("b'--2HUmLkcNHZp0xw6AMBPg'", ratings_train, user_arr, business_arr, business_lookup)

,business_id,name
1045,b'yA8iKH7GNFzA8jgyvjAg3g',"b""Dave & Buster's"""
14781,b'icVcn0oIocwolDnf_CuPQA',b'Thai House'
15935,b'ukCTLs6T8LY1eUBXiAlmCg',b'Steak 44'
16055,b'JzOp695tclcNCNMuBl7oxA',b'Four Peaks Brewing'
18524,b'o69XnnmtOaj6a2sjOTsGeg',"b""Capriotti's Sandwich Shop"""
...,...,...
1267341,b'vXpRQBFvynulGZvXSO9Yhw',b'Voltigeurs'
1338179,b'29qX-E8uPc9PyCz_eVWNjA',b'New York Style Pizza & Deli'
1441802,b'jFIevCnUJG4vuWzSIdhXxQ',b'Dadam'
1539625,b'Jg5a0ltJYBx1tk1kAO0d8w',b'Far East Chinese Food'


In [30]:
def rec_business(user_id, mf_train, user_vecs, business_vecs, user_list, business_list, business_lookup, num_business = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    user_id - Input the user's id that you want to get recommendations for
    
    mf_train - The training matrix used for matrix factorization fitting
    
    user_vecs - the user vectors from fitted matrix factorization
    
    business_vecs - the business vectors from fitted matrix factorization
    
    user_list - an array of the user's IDs that make up the rows of ratings matrix 
    
    business_list - an array of the businesses that make up the columns of ratings matrix
    
    business_lookup - A simple pandas dataframe of the unique business/name description
    
    num_business - The number of businesses you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/business vectors for businesses that were never rated before
    '''
    
    user_ind = np.where(user_list == user_id)[0][0] # Returns the index row of our user id
    pref_vec = mf_train[user_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that business not yet rated become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already rated as zero
    rec_vector = user_vecs[user_ind,:].dot(business_vecs.T) # Get dot product of user vector and the business vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] # makes it an array instead of matrix
    recommend_vector = pref_vec*rec_vector_scaled   # Businesses already rated have their recommendation multiplied by zero
    business_idx = np.argsort(recommend_vector)[::-1][:num_business] # Sort the indices of the business and select top N
    rec_list = [] # start empty list to store items
    for index in business_idx:
        bus_id = business_list[index]
        rec_list.append([bus_id, business_lookup.name.loc[business_lookup.business_id == bus_id].iloc[0]]) 
        # Append our descriptions to the list
    bus_ids = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'business_id': bus_ids, 'name': descriptions}) # Create a dataframe 
    return final_frame[['business_id', 'name']] # Switch order of columns around

In [32]:
rec_business("b'--2HUmLkcNHZp0xw6AMBPg'", ratings_train, user_vecs, business_vecs, user_arr, business_arr, business_lookup, num_business=10)

,business_id,name
0,b'c8rPUIgrXr92LHBvrFQd1Q',b'Yin Ji Chang Fen'
1,b'KOQw4e1rwCY5e0RnmB5rgQ',b'The Grapevine Restaurant & Karaoke Bar'
2,b'znaF77z3hCOqpLNEPhi1yg',"b""McDonald's"""
3,b'h0fQdldYxuG1Utc_ZK1z0A',b'Einstein Bros'
4,b'bUbKPCN1RQcykWHj6jZCig',"b""Wendy's"""
5,b'rwKqYbWkseELVVSAZ3ogpA',"b""Dom's Pizzeria"""
6,b'BD-OOJ8PXSpYTgTHHhXt3g',b'First Watch'
7,b'NMFWunCA1_7-_UoiExtoTA',"b""Quintealia's Tea Parlor"""
8,b'1I8XRD2bf2nfRgjQjgeIOQ',b'7-Eleven'
9,b'eFoKqhHf-jxwKVB69iEbrw',b'Punto Gelato'


## Implement evaluation 